<a href="https://colab.research.google.com/github/jihyeon602/Mental_Health-CHATBOT/blob/main/simple_chatbot_aihub_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 데이터셋 ai-hub: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 로드

In [2]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.37510467 -0.77338433  0.59277105 ...  0.579235    0.32683477
  -0.6508964 ]
 [-0.09361708 -0.18191525 -0.19230807 ... -0.03165793  0.30412546
  -0.26793617]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-006

In [3]:
from openpyxl import load_workbook
df = pd.read_excel('/content/Training.xlsx', engine='openpyxl')

df.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  51630 non-null  int64 
 1   연령          51630 non-null  object
 2   성별          51630 non-null  object
 3   상황키워드       51630 non-null  object
 4   신체질환        51630 non-null  object
 5   감정_대분류      51630 non-null  object
 6   감정_소분류      51630 non-null  object
 7   사람문장1       51630 non-null  object
 8   시스템문장1      51630 non-null  object
 9   사람문장2       51630 non-null  object
 10  시스템문장2      51630 non-null  object
 11  사람문장3       42695 non-null  object
 12  시스템문장3      42695 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.1+ MB


# 전처리

In [5]:
df = df.drop(columns=['Unnamed: 0', '연령', '성별', '신체질환',])

df.head()

,상황키워드,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,"진로,취업,직장",분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,"진로,취업,직장",분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,"진로,취업,직장",분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,"진로,취업,직장",분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,"진로,취업,직장",분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [6]:
df = df.dropna(subset=['시스템문장1', '시스템문장2', '시스템문장3'])

In [10]:
df = df.reset_index(drop=True)
df.head()

,상황키워드,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,"재정,은퇴,노후준비",상처,상처,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.,돈이 없다고 하니 자식이 화를 내서 상처를 받으셨군요.,너무 화가 나. 도와줬던 건 기억 못 하고 더 받을 생각만 하니.,자식들이 받을 생각만 해서 화가 나셨군요. 이 기분을 바꿀 만한 일이 있을까요?,앞으로는 자식보다 나를 생각하며 살아야겠어. 자식 위해 살아도 필요 없다는 말 이제...,이제는 자식보다 나를 생각하며 살려 하시는군요.
1,"재정,은퇴,노후준비",상처,상처,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...,비웃는 친구의 말에 기분이 나쁘셨군요.,아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.,은퇴를 비웃는 친구가 나쁘다고 생각하시는군요. 앞으로 어떻게 하실 생각이신가요?,은퇴 후 조금 쉬며 즐기려고 해. 내가 잘할 수 있는 일이 분명 있을 테니까.,은퇴 후에는 쉬면서 즐기려고 할 생각이시군요.
2,"재정,은퇴,노후준비",상처,상처,친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했어.,친구분이 은퇴를 한심하게 취급해서 서운하셨군요.,맞아. 그동안 내가 얼마나 힘들게 일했는지 알고 있으면서 그런 말을 하다니.,얼마나 힘들었는지 아는 친구분이 그런 말을 했군요. 기분이 나아질 만한 방법이 있을까요?,감정을 추스르고 나중에 연락을 해야겠어. 지금은 많이 서운해.,감정을 추스르고 나중에 연락을 할 생각이시군요.
3,"재정,은퇴,노후준비",상처,상처,그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...,그동안 너무 고생하셨어요. 주변 분들이 많이 걱정해 주시나 봐요.,응. 그렇지만 내 결정에 다들 냉담하게 반응하니 기분이 좋지 않아.,다들 냉담하게 반응해 기분이 좋지 않으시군요. 이 기분을 어떻게 바꿔 볼 수 있을까요?,일단 여행을 다녀 보려고. 그동안 너무 앞만 보고 달려왔어.,여행을 다녀 볼 생각이시군요.
4,"재정,은퇴,노후준비",상처,상처,많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋아.,은퇴를 결심했는데 주변에선 섣부르다고 해서 마음이 안 좋으셨군요.,힘들게 결심했는데 다들 반대하니까 조금 더 고민해 보는 게 좋을까 싶어.,조금 더 고민하는 게 좋을지 생각하시는군요. 어떤 선택을 해야 좋을까요?,한 달 정도 더 고민을 해 보려고. 지금 너무 마음이 지쳤어.,마음이 지쳐 한 달 정도 더 고민해 볼 생각이시군요.


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42695 entries, 0 to 42694
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상황키워드   42695 non-null  object
 1   감정_대분류  42695 non-null  object
 2   감정_소분류  42695 non-null  object
 3   사람문장1   42695 non-null  object
 4   시스템문장1  42695 non-null  object
 5   사람문장2   42695 non-null  object
 6   시스템문장2  42695 non-null  object
 7   사람문장3   42694 non-null  object
 8   시스템문장3  42695 non-null  object
dtypes: object(9)
memory usage: 2.9+ MB


In [12]:
df.loc[0, '사람문장1']

'지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.'

In [13]:
model.encode(df.loc[0, '사람문장1'])

array([ 5.32879867e-02, -4.32924151e-01,  9.77254063e-02, -1.40403211e-01,
       -1.38371671e-02,  1.05805792e-01, -1.22700043e-01,  2.29544878e-01,
       -1.92651190e-02, -6.43874824e-01,  2.39324734e-01,  2.39999682e-01,
       -1.69545919e-01,  1.32054426e-02,  2.40091473e-01, -3.10494930e-01,
        3.12345415e-01, -3.70790184e-01,  2.60245979e-01, -8.31114352e-01,
       -4.62860651e-02, -5.67777097e-01,  2.95227468e-01, -4.30252284e-01,
        1.96394831e-01, -4.87296939e-01,  7.69327655e-02, -2.41090834e-01,
       -4.42292631e-01,  3.54949266e-01,  2.63039798e-01, -5.72234929e-01,
        5.11107929e-02, -1.31320000e-01,  3.65561277e-01, -2.04060018e-01,
       -3.79077196e-02, -4.78561401e-01, -3.44545573e-01,  2.21850142e-01,
       -3.01984668e-01,  2.49042198e-01,  1.96065992e-01,  2.87095010e-01,
       -3.93420637e-01, -5.00292718e-01,  4.44849581e-01,  6.00972950e-01,
       -4.60730404e-01, -2.22037345e-01, -5.44012785e-01,  7.82054141e-02,
        1.88942522e-01,  

# 단일 발화-응답 쌍 추출

In [14]:
# 사용할 컬럼
cols = ['상황키워드', '감정_대분류',
        '사람문장1', '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3']

df = df[cols]  # 필요한 컬럼만 유지

# 단일 발화-응답 쌍 추출
rows = []
for _, row in df.iterrows():
    for i in range(1, 4):
        user_col = f'사람문장{i}'
        bot_col = f'시스템문장{i}'
        if pd.notna(row[user_col]) and pd.notna(row[bot_col]):
            rows.append({
                '상황키워드': row['상황키워드'],
                '감정_대분류': row['감정_대분류'],
                '유저': row[user_col],
                '챗봇': row[bot_col]
            })

# 최종 데이터프레임 생성
df_single = pd.DataFrame(rows)

# 결과 확인
print(df_single.head())


        상황키워드 감정_대분류                                                 유저  \
0  재정,은퇴,노후준비     상처    지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.   
1  재정,은퇴,노후준비     상처               너무 화가 나. 도와줬던 건 기억 못 하고 더 받을 생각만 하니.   
2  재정,은퇴,노후준비     상처  앞으로는 자식보다 나를 생각하며 살아야겠어. 자식 위해 살아도 필요 없다는 말 이제...   
3  재정,은퇴,노후준비     상처  친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...   
4  재정,은퇴,노후준비     상처             아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.   

                                             챗봇  
0                돈이 없다고 하니 자식이 화를 내서 상처를 받으셨군요.  
1  자식들이 받을 생각만 해서 화가 나셨군요. 이 기분을 바꿀 만한 일이 있을까요?  
2                    이제는 자식보다 나를 생각하며 살려 하시는군요.  
3                         비웃는 친구의 말에 기분이 나쁘셨군요.  
4  은퇴를 비웃는 친구가 나쁘다고 생각하시는군요. 앞으로 어떻게 하실 생각이신가요?  


In [15]:
df_single.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128084 entries, 0 to 128083
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   상황키워드   128084 non-null  object
 1   감정_대분류  128084 non-null  object
 2   유저      128084 non-null  object
 3   챗봇      128084 non-null  object
dtypes: object(4)
memory usage: 3.9+ MB


# 유저 대화내용 인코딩

In [16]:
df_single['embedding'] = model.encode(
    df_single['유저'].tolist(),
    batch_size=64,
    show_progress_bar=True
).tolist()

df_single.head()

Batches:   0%|          | 0/2002 [00:00<?, ?it/s]

,상황키워드,감정_대분류,유저,챗봇,embedding
0,"재정,은퇴,노후준비",상처,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.,돈이 없다고 하니 자식이 화를 내서 상처를 받으셨군요.,"[0.05328769609332085, -0.43292436003685, 0.097..."
1,"재정,은퇴,노후준비",상처,너무 화가 나. 도와줬던 건 기억 못 하고 더 받을 생각만 하니.,자식들이 받을 생각만 해서 화가 나셨군요. 이 기분을 바꿀 만한 일이 있을까요?,"[-0.09061889350414276, -0.3955392837524414, -0..."
2,"재정,은퇴,노후준비",상처,앞으로는 자식보다 나를 생각하며 살아야겠어. 자식 위해 살아도 필요 없다는 말 이제...,이제는 자식보다 나를 생각하며 살려 하시는군요.,"[-0.014793764799833298, -0.5315369963645935, -..."
3,"재정,은퇴,노후준비",상처,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...,비웃는 친구의 말에 기분이 나쁘셨군요.,"[-0.08769526332616806, -0.44504624605178833, 0..."
4,"재정,은퇴,노후준비",상처,아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.,은퇴를 비웃는 친구가 나쁘다고 생각하시는군요. 앞으로 어떻게 하실 생각이신가요?,"[-0.4671822488307953, -0.45275911688804626, 0...."


In [17]:
df_single.to_csv('wellness_dataset_my.csv', index=False)

# 간단한 챗봇

In [18]:
text = '요즘 머리가 아프고 너무 힘들어'

embedding = model.encode(text)

In [19]:
df_single['distance'] = df_single['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

,상황키워드,감정_대분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,"재정,은퇴,노후준비",상처,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.,돈이 없다고 하니 자식이 화를 내서 상처를 받으셨군요.,너무 화가 나. 도와줬던 건 기억 못 하고 더 받을 생각만 하니.,자식들이 받을 생각만 해서 화가 나셨군요. 이 기분을 바꿀 만한 일이 있을까요?,앞으로는 자식보다 나를 생각하며 살아야겠어. 자식 위해 살아도 필요 없다는 말 이제...,이제는 자식보다 나를 생각하며 살려 하시는군요.
1,"재정,은퇴,노후준비",상처,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...,비웃는 친구의 말에 기분이 나쁘셨군요.,아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.,은퇴를 비웃는 친구가 나쁘다고 생각하시는군요. 앞으로 어떻게 하실 생각이신가요?,은퇴 후 조금 쉬며 즐기려고 해. 내가 잘할 수 있는 일이 분명 있을 테니까.,은퇴 후에는 쉬면서 즐기려고 할 생각이시군요.
2,"재정,은퇴,노후준비",상처,친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했어.,친구분이 은퇴를 한심하게 취급해서 서운하셨군요.,맞아. 그동안 내가 얼마나 힘들게 일했는지 알고 있으면서 그런 말을 하다니.,얼마나 힘들었는지 아는 친구분이 그런 말을 했군요. 기분이 나아질 만한 방법이 있을까요?,감정을 추스르고 나중에 연락을 해야겠어. 지금은 많이 서운해.,감정을 추스르고 나중에 연락을 할 생각이시군요.
3,"재정,은퇴,노후준비",상처,그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...,그동안 너무 고생하셨어요. 주변 분들이 많이 걱정해 주시나 봐요.,응. 그렇지만 내 결정에 다들 냉담하게 반응하니 기분이 좋지 않아.,다들 냉담하게 반응해 기분이 좋지 않으시군요. 이 기분을 어떻게 바꿔 볼 수 있을까요?,일단 여행을 다녀 보려고. 그동안 너무 앞만 보고 달려왔어.,여행을 다녀 볼 생각이시군요.
4,"재정,은퇴,노후준비",상처,많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋아.,은퇴를 결심했는데 주변에선 섣부르다고 해서 마음이 안 좋으셨군요.,힘들게 결심했는데 다들 반대하니까 조금 더 고민해 보는 게 좋을까 싶어.,조금 더 고민하는 게 좋을지 생각하시는군요. 어떤 선택을 해야 좋을까요?,한 달 정도 더 고민을 해 보려고. 지금 너무 마음이 지쳤어.,마음이 지쳐 한 달 정도 더 고민해 볼 생각이시군요.


In [20]:
df_single.head()

,상황키워드,감정_대분류,유저,챗봇,embedding,distance
0,"재정,은퇴,노후준비",상처,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.,돈이 없다고 하니 자식이 화를 내서 상처를 받으셨군요.,"[0.05328769609332085, -0.43292436003685, 0.097...",0.395167
1,"재정,은퇴,노후준비",상처,너무 화가 나. 도와줬던 건 기억 못 하고 더 받을 생각만 하니.,자식들이 받을 생각만 해서 화가 나셨군요. 이 기분을 바꿀 만한 일이 있을까요?,"[-0.09061889350414276, -0.3955392837524414, -0...",0.336621
2,"재정,은퇴,노후준비",상처,앞으로는 자식보다 나를 생각하며 살아야겠어. 자식 위해 살아도 필요 없다는 말 이제...,이제는 자식보다 나를 생각하며 살려 하시는군요.,"[-0.014793764799833298, -0.5315369963645935, -...",0.296848
3,"재정,은퇴,노후준비",상처,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...,비웃는 친구의 말에 기분이 나쁘셨군요.,"[-0.08769526332616806, -0.44504624605178833, 0...",0.268557
4,"재정,은퇴,노후준비",상처,아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.,은퇴를 비웃는 친구가 나쁘다고 생각하시는군요. 앞으로 어떻게 하실 생각이신가요?,"[-0.4671822488307953, -0.45275911688804626, 0....",0.332968


In [22]:
answer = df_single.loc[df_single['distance'].idxmax()]

print('감정', answer['감정_대분류'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['distance'])

감정 불안
유사한 질문 요즘에 머리가 너무 아파서 걱정이야.
챗봇 답변 요즘에 머리가 너무 아파서 걱정이시군요.무슨 일 있으신가요?
유사도 0.9255012460451634


In [24]:
df_single.tail()

,상황키워드,감정_대분류,유저,챗봇,embedding,distance
128079,대인관계,불안,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,"[-0.5934095978736877, -0.40884777903556824, 0....",0.514486
128080,대인관계,불안,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.,"[0.24901321530342102, -0.3025094270706177, -0....",0.089312
128081,대인관계,상처,남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.,대인관계에 대한 어려움 때문에 걱정되시고 속상하시겠어요.,"[-0.5303046703338623, -0.6839289665222168, -0....",0.239602
128082,대인관계,상처,사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야.,어떻게 하면 지금의 상황에 변화를 만들어낼 수 있을까요?,"[-0.5182764530181885, -0.17631113529205322, 0....",0.293199
128083,대인관계,상처,사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...,원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.,"[-0.08261366933584213, -0.22394530475139618, 0...",0.198753
